This python notebook has exaples of all of the functions we used to get our results with associated tests, if you are looking just for where we reproduce the analytic results look at the results notebook. 

In [1]:
import matplotlib.pyplot as plt 
%matplotlib inline 
import numpy as np 
import math
import time 
from scipy import linalg as la
import copy as cp 

In [14]:
'''
This cell sets up the matrix for non-interacting particles.
This was used almost exactly later for the matrix building function 
'''

n = 200

#(p0,pmax,n)
p = np.linspace(0,10, num = n)


#uses of stepsize are actually defined in terms of 1/h for simplicity
hh = (n/(p[-1] - p[0]))**2
hh2 = 2*hh


#this cuts off the endpoints 
A = np.zeros(shape = (n-1,n-1))
V = np.zeros(n+1)


for i in range(n):
    V[i] = p[i]**2

for i in range(n-1):
        A[i][i] = hh2+V[i+1]
        
#we evaluate v[i+1] so that we avoid the 0 point 
#dont overwrite (n-2)
for i in range(n-2):
        A[i+1][i]= A[i][i+1] = -hh



#test that the matrix is set up correctly, expect 3,7,11,...
evals = la.eigvals(A)


s = np.sort(evals)
print(s[0:4])



[  3.01428603+0.j   7.03122801+0.j  11.04564063+0.j  15.05752148+0.j]


In [15]:
'''
Here we just set up a function to give us the maximum non-diagonal value 
and its indices
'''


def pickmax(a):
    
    #initialize the maximum value and its indices 
    #more notes in the following markdown cell
    big = -1. 
    indi = 0
    indj = 0
    
    #iterate over matrix (input a) to find largest value
    for i in range(len(a)):
        for j in range(i+1,len(a)):
            if abs(a[i][j])> big:
                big = abs(a[i][j])
                indi = i
                indj = j
    return big, indi, indj

#test that we ge the right value and indices, expected 4 with indices 0,1
test = np.array([[2.0,-4.0],[-4.0,1.0]])
pickmax(test)

(4.0, 0, 1)

The above cell has some interesting decisions that help eliminate extra calculations that I would like to mention:
- When initializing the max value for the matrix it was set to -1, this made sure that the condition in the for loops would be proc'd at least once without using >= which would have required an extra test per loop. 
- Because we are working with a symmetric matrix we are able to run the second loop from i+1 to the end of the matrix. This eliminates almost half of the required tests. 

In [25]:
'''
Here we set up a function to work out the values of our Trig funcitons
This is based on the work we did in class
'''


#set the sin,cos and tan for the max value, 
#input: a = nxn matrix,(p,q) are the index outputs of the pickmax function
def settrig(a,p,q):
    
    #initialize tan, sin,l and cos respectively  
    t = s = c = 0
    
    #handles the non-singular case
    if a[p][q] != 0:
        tau = (a[q][q]-a[p][p])/(2*a[p][q])
    
        if tau < 0:
             t = -(-tau+math.sqrt(1+tau**2))**(-1.)
        else: 
            t = (tau+math.sqrt(1+tau**2))**(-1.)
        c = (math.sqrt(1+t**2))**(-1.)
        s = t*c

    else:
        c = 1.0
        s = 0.0
    return t,c,s

#test to make sure the trig solver works expect: ~.88278,~.749678, ~.66180
test = np.array([[2.0,-4.0],[-4.0,1.0]])
settrig(test,0,1)

(0.88278221853731875, 0.7496781758158659, 0.66180256323574027)

In [30]:
'''
This sets up a single iteration of the jacobi method
based on our in class discussion
We also compare with the linalg solver
'''

#1 iteration of jacobi method, input: a = nxn matrix 
def jacobi(a):
    
    #find inputs to jacobi algo 
    maxval, p, q = pickmax(a)
    t,c,s = settrig(a,p,q)
    
    #note effective 'call by value' 
    b = cp.copy(a)
    
    #jacobi iteration 
    for i in range(len(a)):
        if i!=p or i!=q:
            #rotation 
            b[i][p] = a[i][p]*c-a[i][q]*s
            b[i][q] = a[i][q]*c+a[i][p]*s
            #force symmetry
            b[p][i] = b[i][p]
            b[q][i] = b[i][q]
    #force realtions for diagonal vals & maxval
    b[p][p] = a[p][p]*c**2-2*a[p][q]*c*s+a[q][q]*s**2
    b[q][q] = a[p][p]*s**2+2*a[p][q]*c*s+a[q][q]*c**2
    b[p][q] = b[q][p] = 0
    
    return b,maxval
#test that our sigle iteration works on a 2x2 matrix 
#expected out is a 2x2 matrix with diag = ~5.53, -2.5311 
#also an extra output '4' for the largest non-diagonal matrix element
test = np.array([[2.0,-4.0],[-4.0,1.0]])
testvals = la.eigvals(test)

print('Expected eigenvalues from linalg:')
print(testvals)
print('Jacobi solved eigen-values:')
print(jacobi(test))


Expected eigenvalues from linalg:
[ 5.53112887+0.j -2.53112887+0.j]
Jacobi solved eigen-values:
(array([[ 5.53112887,  0.        ],
       [ 0.        , -2.53112887]]), 4.0)


In [29]:

#Full solver, input: nxn matrix, value of tolerance (suggested 10**-5) 
def esolver(matrix, tolerance):
    
    #initailize maxval & gaurentee 1  iteration 
    maxval = tolerance+1
    
    #iterate until tolerable 
    while tolerance <= maxval:
        matrix, maxval = jacobi(matrix )
        
    return matrix


#test that we get the right e-vals
test3 = np.array([[4.,2.,1.],[2.,4.,1.],[1.,1.,4.]])
test3vals = la.eigvals(test3)


print('Expected eigenvalues from linalg:')
print(test3vals)
print('Jacobi solved eigen-values:')
print(esolver(test3,.001))

Expected eigenvalues from linalg:
[ 6.73205081+0.j  2.00000000+0.j  3.26794919+0.j]
Jacobi solved eigen-values:
[[ 2.          0.          0.        ]
 [ 0.          6.73205081  0.        ]
 [ 0.          0.          3.26794919]]


Now that we have a functioning eigenvalue solver
we just need to put together the matrix with the coulomb potential 
added and compare it to the analytic results. 

In [12]:
'''
functionalized version of the matrix intailization from earlier 
with the electron interaction 
added flexibility by making n (the number of matrix elements),
rho-max ,and omega variables 
'''

def makemat(pmax,omega,n):
    #lets us choose rho-max 
    p = np.linspace(0,pmax, num = n)
    
    #these constants are defined in terms of 1/h for simplicity 
    hh = (n/(p[-1] - p[0]))**2
    hh2 = 2*hh
    
    #Initialize the matrix we will operate on and the potential
    A = np.zeros(shape = (n-1,n-1))
    V = np.zeros(n+1)

    #included omega factor and the interaction to the potential,
    #this lets us compare to different analytic results
    for i in range(n):
        V[i] = (omega*p[i])**2+1/p[i]
    
    #Fill the matrix
    for i in range(n-1):
            A[i][i] = hh2+V[i+1]
    for i in range(n-2):
            A[i+1][i]= A[i][i+1] = -hh
            
    return A


#output of the solver can be sloppy, lets clean it up.  
def clean(a):
    evals  = []
    #pick the diagonals
    for i in range(len(a)):
        evals.append(a[i][i])
        
    #sorted for easy reading 
    #and /2 to account for difference from analytic values
    sorted_evals  = np.sort(evals)/2
    
    return sorted_evals

In [31]:
#put it all together to compare to the analytic result 
A = makemat(40,.25,40)
eigenvals = esolver(A,.00001)
cleaned = clean(eigenvals)
print(cleaned) 

[  0.62181583   1.06526775   1.47963271   1.85087367   2.14990337
   2.40874635   2.77068912   3.22976528   3.76667366   4.37568805
   5.05435585   5.80137175   6.61596051   7.49762833   8.44604449
   9.4609788   10.54226605  11.68978461  12.90344296  14.18317098
  15.52891409  16.94062916  18.41828171  19.96184385  21.57129279
  23.24660973  24.98777904  26.79478762  28.66762441  30.60628002
  32.61074643  34.68101672  36.81708493  39.0189459   41.28659509
  43.6200286   46.01925596  48.48604367  51.10945762]


/home/mroosa/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: RuntimeWarning: divide by zero encountered in double_scalars


These are consistent with the analytic findings 
> que the celebrations 
> \\(^0^)/


For fun lets compare the time it takes jacobi to solve the eigenvalue problem to linalg

In [36]:
startJ = time.clock()
esolver(A,.00001)
dTJ = time.clock()-startJ

startL = time.clock()
la.eigvals(A)
dTL = time.clock()-startL

print('Jacobi:',dTJ)
print('Linalg',dTL)

Jacobi: 0.20473100000000066
Linalg 0.0008759999999998769


In the tone of our fearless leader the president:
> "Jacobi can't even work at 1/100'th the speed of linalg. 
> SAD!!! #Fake_eigenvalues"